In [5]:
import time
import numpy as np
import matplotlib.pyplot as plt

from scipy.interpolate import interp1d
import powerbox as pbox

In [ ]:
ks        = np.load(...) # load array of k-values
Pk_lin    = np.load(...) # load linear P(k)
num_samples = len(ks)

splits = 10    # number of augmentations used for the SSL method

In [ ]:
maps_lognormal_all = []
pk_lognormal_all   = []

Npixel  = 100  # Number of grid-points in the box 
BoxSize = 1000 # size of the box to simulate

for i in range(num_samples):
    interp_func = interp1d(np.log(ks[i]), np.log(Pk_lin[i]))
    
    maps_lognormal = []
    pk_lognormal   = []
    
    for j in range(splits):
        pb = pbox.PowerBox(
            N=Npixel,                     
            dim=2,                        
            pk = lambda k: np.exp(interp_func(np.log(k)))/BoxSize,
            boxlength = BoxSize,           
            seed = (splits*i+j),           

        )
        
        # compute lognormal overdensity field from gaussian overdensity field
        delta_g = pb.delta_x()
        var_g = np.var(delta_g)
        rho_ln = np.exp(delta_g - var_g/2)
        
        maps_lognormal.append(rho_ln - 1)
               
    maps_lognormal_all.append(maps_lognormal)
    pk_lognormal_all.append(pk_lognormal)
    